# Análisis historico de la calidad del aire.

In [ ]:
from pyspark.sql import SparkSession
import os
import psycopg
import time 
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
spark = SparkSession.builder \
    .appName("Análisis Histórico") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
pdf = None
for i in range(10):
    try:
        url = os.getenv("DATABASE_URL")
        connection = psycopg.connect(url)
        print("BD conectada con éxito")
        query = 'SELECT * FROM "marts_CalidadAire"'
        pdf = pd.read_sql(query, connection)
        print(f"Datos cargados exitosamente en Pandas: {len(pdf)} filas")
        connection.close()
        break
    except Exception as e:
        print(f"Esperando a la BD... ({e})")
        time.sleep(2)

In [ ]:
if pdf is not None:
    df = spark.createDataFrame(pdf.where(pd.notnull(pdf), None))
else:
    raise Exception("No se pudieron cargar los datos de la base de datos")

In [ ]:
df.printSchema()
df.show(5)